In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pyfmi import load_fmu
import pyfmi
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyfmi import load_fmu
import pyfmi
import numpy as np
import joblib
from tensorflow import keras
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import mean_squared_error
import math
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [2]:
# ## 预测模型

# def predictcool(average_Tdb,week_number,history1,history2,history3):
#     # Load the model for prediction
#     clf = lgb.Booster(model_file='simulation_model.txt')
#     scaler = joblib.load('scaler.pkl')
#     # build the new dataframe
#     data = {
#         'Number_1': [average_Tdb],
#         'Number_2': [week_number],
#         'Number_3': [history1],
#         'Number_4': [history2],
#         'Number_5': [history3]
#     }

#     new_df = pd.DataFrame(data)
#     # standardize the new dataframe
#     X_predcit = scaler.transform(new_df)
#     prediction = clf.predict(X_predcit)
#     value = prediction[0]
#     return value

# def predictsigle(average_Tdb,week_number,history1,history2,history3):
#     # Load the model for prediction
#     clf = lgb.Booster(model_file='simulation_model_sigle.txt')
#     scaler = joblib.load('scaler_sigle.pkl')
#     # build the new dataframe
#     data = {
#         'Number_1': [average_Tdb],
#         'Number_2': [week_number],
#         'Number_3': [history1],
#         'Number_4': [history2],
#         'Number_5': [history3]
#     }

#     new_df = pd.DataFrame(data)
#     # standardize the new dataframe
#     X_predcit = scaler.transform(new_df)
#     prediction = clf.predict(X_predcit)
#     value = prediction[0]
#     return value

# ## 确定稳定后冷机的台数

# def staychiller (q_pre):
    
#     number = math.ceil(q_pre/1759)
#     chiller_number = number/2
#     return chiller_number

In [3]:
## 两个预测模型
def predictcool_rf(average_Tdb, week_number, history1, history2, history3, indoor_temp):
    clf = lgb.Booster(model_file='simulation_model.txt')
    scaler = joblib.load('scaler.pkl')

    # 尝试加载残差修正模型
    try:
        correction_model = joblib.load('correction_model_rf.pkl')
        correction_model_exists = True
    except FileNotFoundError:
        correction_model_exists = False    
    
    # 构建预测数组
    data = {
        'Outdoor_temperature': [average_Tdb],
        'day_of_week': [week_number],
        'history1': [history1],
        'history2': [history2],
        'history3': [history3],
    }

    df = pd.DataFrame(data)
    # 标准化新数据组
    X_predict = scaler.transform(df)
    
    # 使用lightgbm做数据预测
    prediction = clf.predict(X_predict)
    value = prediction[0]
    
    
    # 使用残差修正模型修正预测值
    if correction_model_exists:
        
        # 构建修正预测数组
        new_data = {
            'Outdoor_temperature': [average_Tdb],
            'day_of_week': [week_number],
            'history1': [history1],
            'history2': [history2],
            'history3': [history3],
            'Indoor_temperature': [indoor_temp],
            }
        new_df = pd.DataFrame(new_data)
        correction = correction_model.predict(new_df)
        corrected_value = value + correction[0]
        return corrected_value
    else:
        return value
    
def predictsigle_rf(average_Tdb, week_number, history1, history2, history3, indoor_temp):
    clf = lgb.Booster(model_file='simulation_model_sigle.txt')
    scaler = joblib.load('scaler_sigle.pkl')

    # 尝试加载残差修正模型
    try:
        correction_model = joblib.load('correction_sigle_model_rf.pkl')
        correction_model_exists = True
    except FileNotFoundError:
        correction_model_exists = False    
    
    # 构建预测数组
    data = {
        'Outdoor_temperature': [average_Tdb],
        'day_of_week': [week_number],
        'history1': [history1],
        'history2': [history2],
        'history3': [history3],
    }

    df = pd.DataFrame(data)
    # 标准化新数据组
    X_predict = scaler.transform(df)
    
    # 使用lightgbm做数据预测
    prediction = clf.predict(X_predict)
    value = prediction[0]
    # Ensure the prediction is non-negative
    value = max(2000, value)
    # 使用残差修正模型修正预测值
    if correction_model_exists:
        new_data = {
            'Outdoor_temperature': [average_Tdb],
            'day_of_week': [week_number],
            'history1': [history1],
            'history2': [history2],
            'history3': [history3],
            'Indoor_temperature': [indoor_temp],
            }
        new_df = pd.DataFrame(new_data)        
        correction = correction_model.predict(new_df)
        corrected_value = value + correction[0]
        return corrected_value
    else:
        return value

In [4]:
## 随机森林修正模型

def correctionpred_rf(real_data, pred_data, average_Tdb, week_number, history1, history2, history3, indoor_temp):
    residuals_new = real_data - pred_data

    try:
        historical_data = pd.read_csv('historical_data.csv')
        historical_residuals = np.load('historical_residuals.npy')
    except FileNotFoundError:
        historical_data = pd.DataFrame(columns=['Number_1', 'Number_2', 'Number_3', 'Number_4', 'Number_5', 'Indoor_Temp'])
        historical_residuals = np.array([])

    # Append the new data along with indoor_temp
    historical_data = historical_data.append(
        {
            'Number_1': average_Tdb,
            'Number_2': week_number,
            'Number_3': history1,
            'Number_4': history2,
            'Number_5': history3,
            'Indoor_Temp': indoor_temp
        }, 
        ignore_index=True
    )
    historical_residuals = np.append(historical_residuals, residuals_new)

    if len(historical_data) > 10:
        # Train the correction model with the new feature if data points are more than 20
        correction_model = RandomForestRegressor(n_estimators=100, random_state=42)
        correction_model.fit(historical_data, historical_residuals)
        joblib.dump(correction_model, 'correction_model_rf.pkl')
    
    historical_data.to_csv('historical_data.csv', index=False)
    np.save('historical_residuals.npy', historical_residuals)

    if len(historical_data) > 10:
        return correction_model
    else:
        return None  # Or some other default behavior if not enough data points
    
def correctionsigl_rf(real_data, pred_data, average_Tdb, week_number, history1, history2, history3, indoor_temp):
    residuals_new = real_data - pred_data

    try:
        historical_data = pd.read_csv('historical_sigle_data.csv')
        historical_residuals = np.load('historical_sigle_residuals.npy')
    except FileNotFoundError:
        historical_data = pd.DataFrame(columns=['Number_1', 'Number_2', 'Number_3', 'Number_4', 'Number_5', 'Indoor_Temp'])
        historical_residuals = np.array([])

    # Append the new data along with indoor_temp
    historical_data = historical_data.append(
        {
            'Number_1': average_Tdb,
            'Number_2': week_number,
            'Number_3': history1,
            'Number_4': history2,
            'Number_5': history3,
            'Indoor_Temp': indoor_temp
        }, 
        ignore_index=True
    )
    historical_residuals = np.append(historical_residuals, residuals_new)

    if len(historical_data) > 10:
        # Train the correction model with the new feature if data points are more than 20
        correction_model = RandomForestRegressor(n_estimators=100, random_state=42)
        correction_model.fit(historical_data, historical_residuals)
        joblib.dump(correction_model, 'correction_model_rf.pkl')
    
    historical_data.to_csv('historical_sigle_data.csv', index=False)
    np.save('historical_sigle_residuals.npy', historical_residuals)

    if len(historical_data) > 10:
        return correction_model
    else:
        return None  # Or some other default behavior if not enough data points
    
def staychiller (q_pre):
    
    number = math.ceil(q_pre/1759)
    chiller_number = number/2
    return chiller_number

In [5]:
## 灰箱子模型

## 计算COP数值
# 讲模型此刻的Tcon, Teva,以及计算出来的

def Copcal(PLR,TEVA,TCON):
    # Load the saved model
    pre_model = keras.models.load_model("COP_model.h5")
    # Load the scaler object from the file
    scaler = joblib.load('scaler_cop.pkl')

    data = {
        'Number_1': [PLR],
        'Number_2': [TEVA],
        'Number_3': [TCON],
            }

    new_df = pd.DataFrame(data)
    X_other_standardized = scaler.transform(new_df)
    prediction = pre_model.predict(X_other_standardized)
    Cop_value = prediction[0][0]
    
    return Cop_value

## 计算总能耗

def Energycal(M_room,Q_pre,q_pre,T_room,N_chiler,Cop_value): 
    t_pre = M_room * (T_room - (273.5+24))*1.004 /(N_chiler * 3517)
    t_m = (Q_pre - q_pre*(60 + t_pre ))/ (N_chiler * 3517)
    t_stay = t_pre - t_m
    E_chiller = N_chiler * 732.7 * t_m +  q_pre/Cop_value * t_stay
    E_pump = N_chiler * 34.5 * t_m + N_chiler * 34.5 * t_stay * (q_pre/3517/N_chiler)**3
    E_total = E_chiller + E_pump
    if t_pre > 120:
        E_total = 20000000
    return E_total, t_pre


## 冷机台数参数寻优

def Optichi(M_room,Q_pre, q_pre, T_room, TEVA,TCON):
    min_E_total = float('inf')  # Initialize min_E_total to positive infinity
    best_N_chiller = None
    

    for N_chiller in np.arange(1, 5, 0.5):
        PLR = q_pre/N_chiller/3517
        if PLR > 1.2:
            PLR = 1.2
        elif PLR < 0.2:
            PLR = 0.2
        Cop_value = Copcal(PLR,TEVA,TCON)
        
        E_total, t_pre = Energycal(M_room,Q_pre, q_pre, T_room, N_chiller, Cop_value)
        
        # Check if current E_total is less than the minimum found so far
        if E_total < min_E_total:
            min_E_total = E_total
            best_N_chiller = N_chiller
            best_t_m = t_pre

    return best_N_chiller, best_t_m

## 复查冷机台数优化计算结果

def recheck(M_room,Q_pre, q_pre, T_room, TEVA, TCON):
    Energy_list = []
    Chiller_number = []
    for N_chiller in np.arange(1, 5, 0.5):
        PLR = q_pre / N_chiller / 3517
        if (PLR > 1.2):
            PLR = 1.2
        elif (PLR < 0.2):
            PLR = 0.2
        
        Cop_value = Copcal(PLR, TEVA, TCON)
        E_total, t_pre = Energycal(M_room,Q_pre, q_pre, T_room, N_chiller, Cop_value)
        
        
        Energy_list.append(E_total)
        Chiller_number.append(N_chiller)
    
    return Energy_list

## 根据前一天的结果修订房间总热容

def fixcap(M_room,T_check,T_out):
    value = M_room * (T_out - 297.5)/ (T_out-T_check) 
    return value

In [6]:
# 初始化标志变量
params_updated = False
last_update_day = -1  # 用于记录上次更新的日期
saved_chillern = None
saved_chillern2 = None
saved_time = None

a = 0
stop_t = 86400 * 33  # 运行33天
step_size = 60
model1 = load_fmu("Morning_start_test10.fmu")
model1.initialize()

# 初始化数据存储
cool_load_total_data = []
cool_load_store_data = []
Energy_con = []
outdoor_temperature_data = []
indoor_temperature_recheck = []
indoor_temperature_checkpoint = []
indoor_temperature_data = []  # 存储重命名的indoor_temperature
chiller_number_data = []      # 存储重命名的chiller_number
time_data = []                # 存储时间数据
cop_data = []     #存储系统运行的模型类别
power_data = []  # 存储冷机能耗数据
EVA_temperature = [] # 存储冷机进出口温度
CON_temperature = []

# 记录预测结果
Q_prediction_data = []
q_prediction_data = []

# 记录优化的结果
optimal_chiller = []
optimal_time = []
optimal_chiller_sigle = []

# 记录真数下（非预测）的优化结果
opt_chiller_real = []
opt_time_real = []
opt_chiller_sigle_real = []
opt_consumption = []

# 星期映射表，0代表星期四，1代表星期五，依此类推
week_days = [3, 4, 5, 6, 7, 1, 2]

# 初始天数偏移，确保第一天为星期四
initial_day_offset = 0  # 因为第0天是星期四
MT = 480

## 给定一个初始房间热容值,随后进行修订
M_room = 75000* 2


while a < stop_t:
    status = model1.do_step(a, step_size)
    if status == pyfmi.fmi.FMI_OK:
        # 确保时间的正确性
        date_update = a % 86400
        current_day = (a // 86400 + initial_day_offset) % 7  # 当前是第几天
        week_day = week_days[current_day]  # 当前是星期几

        # 获取室外温度，TEVA, TCON, 室内温度在早上7点
        if date_update == 25200:  # 25200秒 = 7小时 * 3600秒/小时
            outdoor_temperature = model1.get('room_cooling_model.timTab5.y[1]')
            outdoor_temperature = outdoor_temperature[0] - 273.15
            outdoor_temperature_data.append(outdoor_temperature)
            chiller_EVA_temperature = model1.get('mulChi.chi[1].TEvaEnt')
            chiller_CON_temperature = model1.get('mulChi.chi[1].TConEnt')
            EVA_temperature.append(chiller_EVA_temperature[0]+2)
            CON_temperature.append(chiller_CON_temperature[0]+4.5)
            indoor_temperature = model1.get('room_cooling_model.T_room')
            indoor_temperature = indoor_temperature[0]
            indoor_temperature_recheck.append(indoor_temperature - 273.5)
            
            
            # 当天数大于3时操作灰箱模型
            if a > 86400 * 3:
                # 进行预测
                history_total_3 = cool_load_total_data[-3]
                history_total_2 = cool_load_total_data[-2]
                history_total_1 = cool_load_total_data[-1]
                Q_prediction = predictcool_rf(outdoor_temperature, week_day, history_total_1, history_total_2, history_total_3,indoor_temperature - 273.5)
                Q_prediction_data.append(Q_prediction)
                # 11点单次冷负荷预测
                history_sigle_3 = cool_load_store_data[-3]
                history_sigle_2 = cool_load_store_data[-2]
                history_sigle_1 = cool_load_store_data[-1]
                q_prediction = predictsigle_rf(outdoor_temperature, week_day, history_sigle_1, history_sigle_2, history_sigle_3, indoor_temperature - 273.5)
                q_prediction_data.append(q_prediction)
                # 进行优化计算
                best_N_chiller, best_t_m = Optichi(M_room,Q_prediction, q_prediction, indoor_temperature, 
                                                   chiller_EVA_temperature[0]+2, chiller_CON_temperature[0]+4.5)
                optimal_chiller.append(best_N_chiller)
                optimal_time.append(best_t_m)

#                 # 确定稳定状态的制冷机数量
                Stay_number = staychiller(q_prediction)
                optimal_chiller_sigle.append(Stay_number)
                # 确定提前开机的时间
                MT = math.floor(540 - best_t_m)
                model1.set('supper_chiller_number_control.chillern', best_N_chiller)
                model1.set('supper_chiller_number_control.chillern2', Stay_number)



        ## 当目标值达到设定点时修改设定模型的时间参数为1
        if  date_update == MT*60 :
            model1.set('Time_control.k', 1)
            model1.set('Time_control1.k', 1)
            model1.set('Time_control2.k', 1)
            model1.set('Time_control3.k', 1)            
        
        ## 当目标值达到每日的9点整时，将模型时间修改回原先的数值0        
        if  date_update == 32400:
            model1.set('Time_control.k', 0)            
        ## 当目标值达到每日的10点整时，将模型时间修改回原先的数值0        
        if  date_update == 36000:
            model1.set('Time_control.k', 2)
            model1.set('Time_control2.k', 0)
            model1.set('Time_control3.k', 0)
        ## 当目标值达到每日的10点2分时，将模型时间修改回原先的数值0 
        if  date_update == 36120:
            model1.set('Time_control.k', 0)        
        
        ## 当目标值达到每日的下午8点整时，将模型时间修改回原先的数值0 
        if date_update == 72000:
            model1.set("Time_control1.k",0)         
        

        # 在早上9点检查室内温度
        if date_update == 32400:
            room_temperature_check = model1.get('room_cooling_model.T_room')
            indoor_temperature_checkpoint.append(room_temperature_check[0] - 273.15)
        ## 根据9点室内温度值修订房间的热容数值
            M_room = fixcap(M_room,room_temperature_check[0],indoor_temperature)
            
            

        # 从早上7点到早上10点提取indoor_temperature和chiller_number
        if 25200 <= date_update <= 36000 and date_update != MT*60  :
            indoor_temperature_renamed = model1.get('room_cooling_model.T_room')
            indoor_temperature_data.append(indoor_temperature_renamed[0] - 273.15)
            chiller_number = model1.get('chiller_number_control.n')
            chiller_number_data.append(chiller_number[0])
            chiller_cop = model1.get('mulChi.chi[1].COP')
            cop_data.append(chiller_cop[0])
            power_consumption = model1.get('mulSum.y')
            power_data.append(power_consumption[0]*60)
            
            
            # 提取时间并格式化为HH:MM
            hours = (a // 3600) % 24
            minutes = (a // 60) % 60
            time_formatted = f"{hours:02}:{minutes:02}"
            time_data.append(time_formatted)
            
        # 获取冷负荷数据和消耗数据在中午12点
        if date_update == 43200:
            cooling_all = model1.get("Cooling_load1.y")
            cooling_single = model1.get("Cooling_load.y")
            energy_consumption = model1.get("intWitRes1.y")
            cool_load_total_data.append(cooling_all[0] / 60)
            cool_load_store_data.append(cooling_single[0] / 3600)
            Energy_con.append(energy_consumption[0])
            
            ##  修正预测模型
            if  a > 86400 * 3:
                correctionpred_rf(cooling_all[0] / 60, Q_prediction, outdoor_temperature, week_day, 
                                  history_total_1, history_total_2, history_total_3 , indoor_temperature - 273.5)
                
                correctionsigl_rf (cooling_single[0] / 3600, q_prediction, outdoor_temperature, week_day, 
                                   history_sigle_1, history_sigle_2, history_sigle_3 , indoor_temperature - 273.5)            
            
            

#             # 根据真实冷负荷数据重新检查
#             real_opt_chiller, real_opt_t_m = Optichi(M_room,cooling_all[0] / 60, cooling_single[0] / 3600, indoor_temperature,
#                                                      chiller_EVA_temperature[0], chiller_CON_temperature[0])
#             opt_chiller_real.append(real_opt_chiller)
#             opt_time_real.append(real_opt_t_m)
#             # 确定稳定状态的制冷机数量
#             real_Stay_number = staychiller(cooling_single[0] / 3600)
#             opt_chiller_sigle_real.append(real_Stay_number)

#             # 计算不同策略下的实际消耗
#             re_calculate = recheck(M_room,cooling_all[0] / 60, cooling_single[0] / 3600, indoor_temperature,
#                                    chiller_EVA_temperature[0], chiller_CON_temperature[0])
#             opt_consumption.append(re_calculate)

        a = a + step_size
    else:
        print("error")
        break

In [7]:
check_show75 = pd.DataFrame({"indoor_temperature":indoor_temperature_data,'time':time_data,
                              "chiller_number":chiller_number_data,"chiller_cop": cop_data ,'power_consumption':power_data})

In [8]:
check_show75

,indoor_temperature,time,chiller_number,chiller_cop,power_consumption
0,28.005688,07:00,0.5,0.000000,1.317259
1,28.005724,07:01,0.5,0.000000,1.316095
2,28.005760,07:02,0.5,0.000000,1.314922
3,28.005796,07:03,0.5,0.000000,1.313739
4,28.005832,07:04,0.5,0.000000,1.312547
...,...,...,...,...,...
5935,24.200721,09:56,1.0,5.677394,5.672818
5936,24.206733,09:57,1.0,5.683126,5.668842
5937,24.212079,09:58,1.0,5.689050,5.664818
5938,24.216155,09:59,1.0,5.695344,5.660706


In [9]:
check_show75.to_csv('room_temperature_numberOP729.csv')

In [10]:
consumption_data = pd.DataFrame({"Consumption":Energy_con,"indoor temperature":indoor_temperature_checkpoint})
consumption_data.to_csv('consumption_OP729.csv')

In [11]:
cooling_total = cool_load_total_data[3:]
cooling_sigel = cool_load_store_data[3:]
indoor_temperature_list = indoor_temperature_recheck[3:]


In [12]:
EVA_temperature_list = EVA_temperature[3:]
CON_temperature_list = CON_temperature[3:]

In [13]:
predict_check725 = pd.DataFrame({"indoor_temperature":indoor_temperature_list,"real_Q":cooling_total,"real_q":cooling_sigel,
                                 "prediction_Q":Q_prediction_data,"prediction_q":q_prediction_data,'EVA_T':EVA_temperature_list,'CON_T':CON_temperature_list})

In [14]:
predict_check725.to_csv('prediction_result_729.csv')